##### Google PaLM LLM model and API KEY setup 

In [1]:
from langchain.llms import GooglePalm

api_key = 'Your own API KEY'

llm = GooglePalm(google_api_key=api_key, temperature=0.2) # temperature ca take value between (0 - 1) (0 = less creative , 1 = most creative)
poem = llm("Write a poem on for my love for Bangladesh")
print(poem)

c:\Users\m2hasan\AppData\Local\anaconda3\envs\LLM_Sql\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


My Bangladesh, my beloved land,
Your beauty is beyond compare.
Your lush green hills and valleys,
Your golden beaches and coral reefs,
Your mighty rivers and sparkling lakes,
All make you a sight to behold.

Your people are warm and welcoming,
Your culture is rich and vibrant,
Your history is long and proud.
You are a land of contrasts,
From the bustling city of Dhaka
To the sleepy villages of the countryside.

But through it all, you are always Bangladesh,
My home, my heart, my everything.
I love you Bangladesh,
With all my heart.

I love your warm, sunny days,
And your cool, refreshing nights.
I love your delicious food,
And your friendly people.

I love your beautiful culture,
And your rich history.
I love your vibrant spirit,
And your endless potential.

I love you, Bangladesh,
With all my heart.

You are my home,
And I will always love you.


##### Connect with MySQL Database and see database info

In [3]:
from langchain.utilities import SQLDatabase

db_user = "root"
db_password = "root"
db_host ="localhost"
db_name = "atliq_tshirts"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}", sample_rows_in_table_info=3)
print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

##### Performing some query in the database

In [4]:
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm, db, verbose = True) #Verbose = True, shows us all the intermediate steps to the results.

#Sample Question 1:
qns1 = db_chain.run("How many t-shirts do we have left for nike in extra small size and white color?")



> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for nike in extra small size and white color?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'
SQLResult: [(74,)]
Answer:74
> Finished chain.


In [5]:
qns1

'74'

 👍🏼 Answer for the question 1 is correct.

In [5]:
#Sample Question 2:
qns2 = db_chain.run("How much is the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT SUM(price) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('386'),)]
Answer:386
> Finished chain.


It makes a mistake here, in out database the price coulumn in the t_shirts table is the price of per unit(tshirt). However, LLM understands that as the price of all the Small size t-shirts. As a result, it only sums up the prices and forgot to multiply the price with stock_quantity before doing the sum. The correct query sould be "SELECT sum(stock_quantity*price) FROM t_shirts WHERE size = 'S'"

In [6]:
#correct query for question 2
qns2 = db_chain.run("SELECT sum(stock_quantity*price) FROM t_shirts WHERE size = 'S'")



> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity*price) FROM t_shirts WHERE size = 'S'
SQLQuery:SELECT sum(stock_quantity*price) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('16235'),)]
Answer:16235
> Finished chain.


As the LLM model makes the mistake we need to update it's understandings. The process of updating it's knowledge is "Few Short Learning". We will use qns2 value later on in this notebook for this learning. So hold on for now and let's check another query.

In [7]:
#correct answer for question 2
qns2

'16235'

In [8]:
#Sample Question 3:
qns3 = db_chain.run("If we have to sell all the Levi's T-Shirts today with discounts applied. How much revenue our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Levi's T-Shirts today with discounts applied. How much revenue our store will generate (post discounts)?
SQLQuery:SELECT SUM(price * (1 - pct_discount)) FROM discounts JOIN t_shirts ON discounts.t_shirt_id = t_shirts.t_shirt_id WHERE brand = 'Levi' AND CURDATE() BETWEEN start_date AND end_date

OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 'start_date' in 'where clause'")
[SQL: SELECT SUM(price * (1 - pct_discount)) FROM discounts JOIN t_shirts ON discounts.t_shirt_id = t_shirts.t_shirt_id WHERE brand = 'Levi' AND CURDATE() BETWEEN start_date AND end_date]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

The result above returns an error as the LLM model returns a wrong query. If we look closely in to the generated query we can see that it says "BETWEEN discounts.start_date AND discounts.end_date". However, our discount table holds the discount in parcentages and there is no start or end date. In general practice it is very common to have a start and end date for the discouns as it sould not provide discount for infinite amount of time. But, our use case is different. LLM uses the common practices, thus it fails in our usecase. So we need to add this to the "Few Short Learning" as well. The correct sql query asn ans is stored for this purpose. 

In [9]:
# Correct query for question 3
qns3_sql_query = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """

qns3 = db_chain.run(qns3_sql_query)



> Entering new SQLDatabaseChain chain...

select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 
SQLQuery:select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('19410.300000'),)]
Answer:19410.3
> Finished chain.


In [10]:
# Correct ans for question 3
qns3

'19410.3'

In [11]:
# Sample Question 4:
qns4 = db_chain.run("If I sell all levi t-shirt, how much revenue I will generate?")



> Entering new SQLDatabaseChain chain...
If I sell all levi t-shirt, how much revenue I will generate?
SQLQuery:SELECT SUM(price) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('411'),)]
Answer:411
> Finished chain.


Again, it makes the same mistake like question 2. It forgots to multiply the price with stock_quantity.

In [12]:
# Correct query for question 4
qns4 = db_chain.run("SELECT SUM(price*stock_quantity) FROM t_shirts WHERE brand = 'Levi'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('20348'),)]
Answer:20348
> Finished chain.


In [13]:
# Correct answer for question 4
qns4

'20348'

In [14]:
# Sample question 5:
qns5 = db_chain.run("How many white color Levi's t shirts we have available?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's t shirts we have available?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(24,), (87,), (91,), (16,)]
Answer:24
> Finished chain.


This output is interesting. If we notice that our database has Levi's white color t-shirts of 4 different sizes (XS,S, M, L). Thus, the LLM model comes up with stock_quantity of each sizes of the Levi's t-shirts. Eventually, it forgot to sum up the results.  

In [15]:
# Correct query for question 5:
qns5 = qns5 = db_chain.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")



> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('218'),)]
Answer:218
> Finished chain.


In [16]:
# Correct answer for question 5:
qns5

'218'

##### Few Short Learning

This is a technique to train LLM models according to your need. In this process we will tackle down all the problems we have already encountered. Besides, it is worth mentioning that we can add as much as short learning into our LLM model. Let's see how to do it. 

In [17]:
# Making a list for short learnings based on our requirements. Here we keep individual learning components inside a dictionary. 

few_shots = [
    {'Question' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns1},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': qns2},
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': qns3} ,
     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : qns4},
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns5
     }
]

##### Creating Semantic Similarity Based example selector
After creating the few short learnings list, we need to perform the following steps. 

* create embedding on the few_shots
* Store the embeddings in Chroma DB
* Retrieve the the top most Semantically close example from the vector store

In [18]:
from langchain.embeddings import HuggingFaceEmbeddings 

# Embedding on the Few_Shorts
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

# Cereating block of texts from the few_Shorts list
to_vectorize = [" ".join(example.values()) for example in few_shots]
to_vectorize

["How many t-shirts do we have left for Nike in XS size and white color? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS' Result of the SQL query 74",
 "How much is the total price of the inventory for all S-size t-shirts? SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S' Result of the SQL query 16235",
 "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)? SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from\n(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'\ngroup by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id\n  Result of the SQL query 19410.3",
 "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount? SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi' 

In [19]:
from langchain.vectorstores import Chroma

# Store the embeddings in Chroma DB 
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots)

In [20]:
from langchain.prompts import SemanticSimilarityExampleSelector

# Retrieve the the top most Semantically close example from the vector store
example_selector= SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2 
)
example_selector.select_examples({"Question": "How many Adidas T shirts I have left in my store?"})


[{'Answer': '74',
  'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': '20348',
  'Question': 'If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?',
  'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
  'SQLResult': 'Result of the SQL query'}]

##### Supply Custom Promt to LLM so that it can perform better

Generally, a promt is something which we send to the LLM model and based on that it gives us the best results. Here we are generating some additional lines for the promt. This lines can be any custom text according to our requiremnets. For example, we are dealing with MySql database here. So it is very logical to say the LLM model to act like a MySQL database expert. Also we can specify any oter requirements for the LLM model. 

This promt text can be written by the developers. However,LangChain library provides an automatic promt for the MySQL database. Here I am going to use that. 

In [25]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt 

print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [26]:
print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


##### Setting up PromptTemplete using input variables

In [27]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [30]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=_mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

##### Creating New chain with the Few Short Learning

In [31]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

##### Let's try the questions again with wrong answers. 

In [34]:
new_chain.run("How many white color Levi's shirt I have?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's shirt I have?
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('218'),)]
Answer:218
> Finished chain.


'218'

In [35]:
new_chain.run("How much is the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT sum(price * stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('16235'),)]
Answer:16235
> Finished chain.


'16235'

In [36]:
new_chain.run("How much is the price of all white color levi t shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of all white color levi t shirts?
SQLQuery:SELECT sum(price*stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('5952'),)]
Answer:5952
> Finished chain.


'5952'

In [37]:
new_chain.run("If we have to sell all the Nike's T-Shirts today with discounts applied. How much revenue our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Nike's T-Shirts today with discounts applied. How much revenue our store will generate (post discounts)?
SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Nike'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('19353.600000'),)]
Answer:19353.6
> Finished chain.


'19353.6'

In [39]:
new_chain.run("If we have to sell all the Van Heuson T-Shirts today with discounts applied. How much revenue our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Van Heuson T-Shirts today with discounts applied. How much revenue our store will generate (post discounts)?
SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Van Huesen'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('26689.600000'),)]
Answer:26689.6
> Finished chain.


'26689.6'

In [40]:
new_chain.run('How much revenue  our store will generate by selling all Van Heuson TShirts without discount?')



> Entering new SQLDatabaseChain chain...
How much revenue  our store will generate by selling all Van Heuson TShirts without discount?
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Van Huesen'
SQLResult: [(Decimal('26887'),)]
Answer:26887
> Finished chain.


'26887'